# W205 Fall Project 2 - Tracking User Activity

## Nobu Yamaguchi

# *Executive summary*

> ## The purpose of this analysis is to show the data scientists three basic business questions to answer about these data.

### Question 1: How many assessment are there?

First of all, it is very important to know how many assessments in the dataset? 

### Answer 1. There are 3280 assessments.

Next, in order to understand the overall trend, I answer the following two questions

### Question 2-1: What are the top 10 popular exams?

### Answer 2-1. 
```
+-----------------------------------------------------------+-----+
|exam_name                                                  |count|
+-----------------------------------------------------------+-----+
|Learning Git                                               |394  |
|Introduction to Python                                     |162  |
|Introduction to Java 8                                     |158  |
|Intermediate Python Programming                            |158  |
|Learning to Program with R                                 |128  |
|Introduction to Machine Learning                           |119  |
|Software Architecture Fundamentals Understanding the Basics|109  |
|Beginning C# Programming                                   |95   |
|Learning Eclipse                                           |85   |
|Learning Apache Maven                                      |80   |
+-----------------------------------------------------------+-----+
```

The table above shows the top 10 popular exams. The most popular one is "Learning Git" with 394 count. Also, there are two python exams in the top 10 list. One is "Introducgtion to Python" and the other is "Intermediate Python Programming".

### Question 2-2: What are the least popular exams (the count is less than 4)?

### Answer 2-2.
```
+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Learning to Visualize Data with D3.js            |1    |
|Native Web Apps for Android                      |1    |
|Nulls, Three-valued Logic and Missing Information|1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
|Learning Spring Programming                      |2    |
|Client-Side Data Storage for Web Developers      |2    |
|Hibernate and JPA Fundamentals                   |2    |
|Arduino Prototyping Techniques                   |2    |
|The Closed World Assumption                      |2    |
|Understanding the Grails 3 Domain Model          |2    |
|What's New in JavaScript                         |2    |
|Service Based Architectures                      |3    |
|Mastering Web Views                              |3    |
|Building Web Services with Java                  |3    |
|Getting Ready for Angular 2                      |3    |
|Using Web Components                             |3    |
+-------------------------------------------------+-----+

```

### Question 3: What are the avarage grades with standard deviation of the 5 top most popular exams?

### Answer 3:
```
+-------------------------------+-------------+---------+
|exam_name                      |average_grade|std_grade|
+-------------------------------+-------------+---------+
|Learning Git                   |68.0         |31.59    |
|Introduction to Python         |57.0         |26.51    |
|Introduction to Java 8         |88.0         |18.78    |
|Intermediate Python Programming|51.0         |29.88    |
|Learning to Program with R     |54.0         |33.07    |
+-------------------------------+-------------+---------+
```

The overall average is 63. "Learning Git" and "Introductory to Jave 8" have high average.

# Codes

In [1]:
import json
import pprint
import sys
from pyspark.sql import Row

In [2]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [3]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [4]:
raw_assessments.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

In [6]:
assessments.write.parquet("/tmp/assessments")

```
$ docker-compose exec cloudera hadoop fs -ls /tmp/
Found 3 items
drwxr-xr-x   - root   supergroup          0 2019-10-21 22:04 /tmp/assessments
drwxrwxrwt   - mapred mapred              0 2018-02-06 18:27 /tmp/hadoop-yarn
drwx-wx-wx   - root   supergroup          0 2019-10-21 21:46 /tmp/hive
```

```
$ docker-compose exec cloudera hadoop fs -ls /tmp/assessments
Found 2 items
-rw-r--r--   1 root supergroup          0 2019-10-21 22:04 /tmp/assessments/_SUCCESS
-rw-r--r--   1 root supergroup    2513397 2019-10-21 22:04 /tmp/assessments/part-00000-9f8e1619-b84d-41a7-be5f-ab17f83282ce-c000.snappy.parquet
```

In [7]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [8]:
extracted_assessments.registerTempTable('assessments')

#### Q1. How many assessment are there?

In [10]:
spark.sql("select count(*) as count from assessments").show()

+-----+
|count|
+-----+
| 3280|
+-----+



#### Question 2-1: What are the top 10 popular exams?

In [14]:
spark.sql("select exam_name, count(*) as count from assessments group by exam_name order by count desc").show(10, False)

+-----------------------------------------------------------+-----+
|exam_name                                                  |count|
+-----------------------------------------------------------+-----+
|Learning Git                                               |394  |
|Introduction to Python                                     |162  |
|Introduction to Java 8                                     |158  |
|Intermediate Python Programming                            |158  |
|Learning to Program with R                                 |128  |
|Introduction to Machine Learning                           |119  |
|Software Architecture Fundamentals Understanding the Basics|109  |
|Beginning C# Programming                                   |95   |
|Learning Eclipse                                           |85   |
|Learning Apache Maven                                      |80   |
+-----------------------------------------------------------+-----+
only showing top 10 rows



#### Question 2-2: What are the least popular exams (the count is less than 4)?

In [15]:
spark.sql("select exam_name, count(*) as count from assessments group by exam_name having count(*) < 4 order by count").show(20, False)

+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Learning to Visualize Data with D3.js            |1    |
|Native Web Apps for Android                      |1    |
|Nulls, Three-valued Logic and Missing Information|1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
|Learning Spring Programming                      |2    |
|Client-Side Data Storage for Web Developers      |2    |
|Hibernate and JPA Fundamentals                   |2    |
|Arduino Prototyping Techniques                   |2    |
|The Closed World Assumption                      |2    |
|Understanding the Grails 3 Domain Model          |2    |
|What's New in JavaScript                         |2    |
|Service Based Architectures                      |3    |
|Mastering Web Views                              |3    |
|Building Web Services with Java                  |3    |
|Getting Ready

#### Question 3: What are the avarage grades of the 5 top most popular exams?

In [18]:
def my_lambda_grades(x):
    raw_dict = json.loads(x.value)
    my_list = []
    
    for l in raw_dict:
        exam_name = raw_dict["exam_name"]
        if "sequences" in raw_dict:
            if "counts" in raw_dict["sequences"]:
                if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    my_correct = raw_dict["sequences"]["counts"]["correct"]
                    my_total = raw_dict["sequences"]["counts"]["total"]
                    my_dict = {"exam_name" : raw_dict["exam_name"], "correct" : my_correct, "total" : my_total}
                    my_list.append(Row(**my_dict))
    return my_list

my_grades = assessments.rdd.flatMap(my_lambda_grades).toDF()

my_grades_2 = my_grades.withColumn("grade", my_grades["correct"]/my_grades["total"]*100)

my_grades_2.registerTempTable('grades')

spark.sql("select exam_name, round(avg(grade)) as average_grade, round(stddev_samp(grade),2) as std_grade from grades group by exam_name order by count(exam_name) desc").show(5, False)

+-------------------------------+-------------+---------+
|exam_name                      |average_grade|std_grade|
+-------------------------------+-------------+---------+
|Learning Git                   |68.0         |31.59    |
|Introduction to Python         |57.0         |26.51    |
|Introduction to Java 8         |88.0         |18.78    |
|Intermediate Python Programming|51.0         |29.88    |
|Learning to Program with R     |54.0         |33.07    |
+-------------------------------+-------------+---------+
only showing top 5 rows



#### Below shows the avarage of total.

In [19]:
spark.sql("select  round(avg(grade)) as average_grade from grades").show()

+-------------+
|average_grade|
+-------------+
|         63.0|
+-------------+



## Appendix

In [133]:
spark.sql("select exam_name, round(avg(grade)) as average_grade, round(stddev_samp(grade),2) as std_grade from grades group by exam_name order by average_grade desc").show(10, False)

+-----------------------------------------------------------------------+-------------+---------+
|exam_name                                                              |average_grade|std_grade|
+-----------------------------------------------------------------------+-------------+---------+
|Learning to Visualize Data with D3.js                                  |100.0        |0.0      |
|Nulls, Three-valued Logic and Missing Information                      |100.0        |0.0      |
|The Closed World Assumption                                            |100.0        |0.0      |
|Learning SQL for Oracle                                                |98.0         |7.22     |
|Introduction to Java 8                                                 |88.0         |18.78    |
|Introduction to Apache Spark                                           |83.0         |16.76    |
|Introduction to Amazon Web Services (AWS) - EC2 Deployment Fundamentals|83.0         |25.67    |
|Getting Ready for A

In [2]:
p = pprint.PrettyPrinter(indent=1)

In [3]:
f = open("assessment-attempts-20180128-121051-nested.json","r")

In [4]:
s = f.read()

In [5]:
json_data = json.loads(s)

In [6]:
f.close()

In [7]:
len(json_data)

3280

In [8]:
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [9]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)

In [10]:
recursive_walk_json_object(json_data[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

In [11]:
!cat assessment-attempts-20180128-121051-nested.json | jq ' .[] | .exam_name' -c | sort | uniq -c | sort -r

    394 "Learning Git"
    162 "Introduction to Python"
    158 "Introduction to Java 8"
    158 "Intermediate Python Programming"
    128 "Learning to Program with R"
    119 "Introduction to Machine Learning"
    109 "Software Architecture Fundamentals Understanding the Basics"
     95 "Beginning C# Programming"
     85 "Learning Eclipse"
     80 "Learning Apache Maven"
     79 "Beginning Programming with JavaScript"
     77 "Mastering Git"
     75 "Introduction to Big Data"
     67 "Advanced Machine Learning"
     59 "Learning Linux System Administration"
     58 "JavaScript: The Good Parts Master Class with Douglas Crockford"
     57 "Learning SQL"
     53 "Practical Java Programming"
     52 "HTML5 The Basics"
     51 "Python Epiphanies"
     48 "Software Architecture Fundamentals Beyond The Basics"
     43 "Introduction to Data Science with R"
     43 "Intermediate C# Programming"
     40 "Learning DNS"
     35 "Learning C# Best Practices"
     35 "Expert Data Wrangling with R"
 

In [12]:
first_message=json.loads(assessments.select('value').take(1)[0].value)

In [13]:
first_message

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
  'counts': {'all_correct': False,
   'correct': 2,
   'incomplete': 1,
   'incorrect': 1,
   'submitted': 4,
   'total': 4,
   'unanswered': 0},
  'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
  'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
    'options': [{'at': '2018-01-23T14:23:24.670Z',
      'checked': True,
      'correct': True,
      'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d',
      'submitted': 1},
     {'at': '2018-01-23T14:23:25.914Z',
      'checked': True,
      'correct': True,
      'id': 'f2528210-35c3-4320-acf3-9056567ea19f',
      'submitted': 1},
     {'checked': False,
      'correct': True,
      'id': 'd1bf026f-554f-4543-bdd2-54dcf10